In [9]:
import os
os.chdir(r"C:\Users\anyaa\Documents\nlp\quora")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from scipy.sparse import hstack,vstack
import contractions
df=pd.read_csv("questions.csv")
df.head(3)
df=df.dropna()
df.isnull().sum()


id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
# Lowercase (vectorized → fast)
df['question1'] = df['question1'].str.lower()
df['question2'] = df['question2'].str.lower()
# Remove special characters (vectorized → fast)
df['question1'] = df['question1'].str.replace(r"[^a-zA-Z0-9\s]", " ", regex=True)
df['question2'] = df['question2'].str.replace(r"[^a-zA-Z0-9?!\s]", " ", regex=True)
# Remove extra spaces (vectorized → fast)
df['question1'] = df['question1'].str.replace(r"\s+", " ", regex=True).str.strip()
df['question2'] = df['question2'].str.replace(r"\s+", " ", regex=True).str.strip()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df[['question1','question2']],
    df['is_duplicate'],
    test_size=0.2,
    random_state=42
)

In [12]:
from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer("all-MiniLM-L6-v2")

q1_emb_train = st_model.encode(
    X_train["question1"].tolist(),
    batch_size=64,
    show_progress_bar=True
)

q2_emb_train = st_model.encode(
    X_train["question2"].tolist(),
    batch_size=64,
    show_progress_bar=True
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\anyaa\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anyaa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5055 [00:00<?, ?it/s]

Batches:   0%|          | 0/5055 [00:00<?, ?it/s]

In [16]:
import numpy as np
from numpy.linalg import norm

trans_cos_train = (
    np.sum(q1_emb_train * q2_emb_train, axis=1) /
    (norm(q1_emb_train, axis=1) * norm(q2_emb_train, axis=1) + 1e-9)
)
X_train["transformer_cosine"] = trans_cos_train
train_dist = norm(q1_emb_train - q2_emb_train, axis=1)
X_train["transformer_cosine_dist"] = train_dist
from scipy.sparse import csr_matrix

X_final = csr_matrix(np.hstack([q1_emb_train , q2_emb_train]))
# Precompute char lengths once
l1 = X_train['question1'].str.len()
l2 = X_train['question2'].str.len()

# Assign
X_train['lencq1'] = l1
X_train['lencq2'] = l2
X_train['lendiff'] = l1 - l2

# Word lengths
X_train['lenq1'] = X_train['question1'].str.split().str.len()
X_train['lenq2'] = X_train['question2'].str.split().str.len()

#common words
q1_words = X_train['question1'].str.split()
q2_words = X_train['question2'].str.split()
X_train['common_word_count'] = [
    len(set(a) & set(b)) for a,b in zip(q1_words, q2_words)
]
#jaccard similarity
def jaccard(q1, q2):
    w1 = set(str(q1).split())
    w2 = set(str(q2).split())
    if len(w1.union(w2)) == 0:
        return 0
    return len(w1.intersection(w2)) / len(w1.union(w2))

X_train['jaccard'] = [
    len(set(a.split()) & set(b.split())) /
    len(set(a.split()) | set(b.split()))
    if len(set(a.split()) | set(b.split())) > 0 else 0
    for a,b in zip(X_train['question1'], X_train['question2'])
]

from scipy.sparse import csr_matrix
numeric_cols = ['lenq1','lenq2','lencq1','lencq2','lendiff','common_word_count','jaccard','transformer_cosine',"transformer_cosine_dist"]
num_train = csr_matrix(X_train[numeric_cols].to_numpy())

X_final_train = hstack([X_final, num_train])


In [ ]:
q1_emb_test = st_model.encode(
    X_test["question1"].tolist(),
    batch_size=64,
    show_progress_bar=True
)

q2_emb_test = st_model.encode(
    X_test["question2"].tolist(),
    batch_size=64,
    show_progress_bar=True
)


Batches:   0%|          | 0/1264 [00:00<?, ?it/s]

In [17]:


X_test['lenq1'] = X_test['question1'].str.split().str.len()
X_test['lenq2'] = X_test['question2'].str.split().str.len()

X_test['lencq1']=X_test['question1'].str.len()
X_test['lencq2']=X_test['question2'].str.len()
X_test['lendiff']=X_test['question1'].str.len()-X_test['question2'].str.len()
def common_words(q1, q2):
    w1 = set(str(q1).lower().split())
    w2 = set(str(q2).lower().split())
    return len(w1.intersection(w2))
X_test['common_word_count'] = [
    len(set(a.split()) & set(b.split()))
    for a,b in zip(X_test['question1'], X_test['question2'])
]

def jaccard(q1, q2):
    w1 = set(str(q1).lower().split())
    w2 = set(str(q2).lower().split())
    if len(w1.union(w2)) == 0:
        return 0
    return len(w1.intersection(w2)) / len(w1.union(w2))
X_test['jaccard'] = X_test.apply(
    lambda row: jaccard(row['question1'], row['question2']),
    axis=1
)



trans_cos_test = (
    np.sum(q1_emb_test * q2_emb_test, axis=1) /
    (norm(q1_emb_test, axis=1) * norm(q2_emb_test, axis=1) + 1e-9)
)

X_test["transformer_cosine"] = trans_cos_test
test_dist = norm(q1_emb_test - q2_emb_test, axis=1)
X_test["transformer_cosine_dist"] = test_dist

from scipy.sparse import csr_matrix

X_final_t = csr_matrix(np.hstack([q1_emb_test , q2_emb_test]))


numeric_cols = [
    'lenq1',
    'lenq2',
    'lencq1','lencq2','lendiff',
    'common_word_count',
    'jaccard',
    'transformer_cosine',
    'transformer_cosine_dist'
]
from scipy.sparse import csr_matrix

num_test_sparse = csr_matrix(X_test[numeric_cols].values)
from scipy.sparse import hstack

X_final_test = hstack([X_final_t, num_test_sparse])

In [18]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=63,
    colsample_bytree=0.8,
    max_depth=-1,
    random_state=42
)

model.fit(X_final_train, y_train)
from sklearn.metrics import accuracy_score, f1_score
y_pred = model.predict(X_final_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))


[LightGBM] [Info] Number of positive: 119676, number of negative: 203802
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.704054 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 197516
[LightGBM] [Info] Number of data points in the train set: 323478, number of used features: 777
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.369966 -> initscore=-0.532361
[LightGBM] [Info] Start training from score -0.532361


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.8683689872635093
F1: 0.8239127917555787
